In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import imutils
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout

In [3]:
class SudokuNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)

        model.add(Conv2D(32, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(64, (3, 3), padding="same"))  # Additional Conv2D layer
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        model.add(Dense(64))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        model.add(Dense(classes))
        model.add(Activation("softmax"))

        return model

In [4]:
model_path = '/content/drive/MyDrive/your_model.h5'
INIT_LR = 1e-3
EPOCHS = 750
BS = 128
print("[INFO] accessing MNIST...")
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
testData = testData.reshape((testData.shape[0], 28, 28, 1))
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0
le = LabelBinarizer()
trainLabels = le.fit_transform(trainLabels)
testLabels = le.transform(testLabels)
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)
model = SudokuNet.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
print("[INFO] training network...")
H = model.fit(
	trainData, trainLabels,
	validation_data=(testData, testLabels),
	batch_size=BS,
	epochs=EPOCHS,
	verbose=1)
print("[INFO] evaluating network...")
predictions = model.predict(testData)
print(classification_report(
	testLabels.argmax(axis=1),
	predictions.argmax(axis=1),
	target_names=[str(x) for x in le.classes_]))
print("[INFO] serializing digit model...")
model.save(model_path, save_format="h5")

[INFO] accessing MNIST...
11490434/11490434 [==============================] - 0s 0us/step
[INFO] compiling model...
[INFO] training network...
Epoch 1/750
 43/469 [=>............................] - ETA: 1:04 - loss: 2.1794 - accuracy: 0.2080

KeyboardInterrupt: 

In [ ]:
def find_puzzle(image, debug=False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (7, 7), 3)
    thresh = cv2.adaptiveThreshold(blurred, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    thresh = cv2.bitwise_not(thresh)
    if debug:
        cv2_imshow(thresh)
        cv2.waitKey(0)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    puzzleCnt = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            puzzleCnt = approx
            break
    if puzzleCnt is None:
        raise Exception(("Could not find Sudoku puzzle outline. "
                "Try debugging your thresholding and contour steps."))
    if debug:
        output = image.copy()
        cv2.drawContours(output, [puzzleCnt], -1, (0, 255, 0), 2)
        cv2_imshow(output)
        cv2.waitKey(0)
    puzzle = four_point_transform(image, puzzleCnt.reshape(4, 2))
    warped = four_point_transform(gray, puzzleCnt.reshape(4, 2))
    if debug:
        cv2_imshow(puzzle)
        cv2.waitKey(0)
    return (puzzle, warped)

def extract_digit(cell, debug=False):
    thresh = cv2.threshold(cell, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    thresh = clear_border(thresh)
    if debug:
        cv2_imshow(thresh)
        cv2.waitKey(0)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    if len(cnts) == 0:
        return None
    c = max(cnts, key=cv2.contourArea)
    mask = np.zeros(thresh.shape, dtype="uint8")
    cv2.drawContours(mask, [c], -1, 255, -1)
    (h, w) = thresh.shape
    percentFilled = cv2.countNonZero(mask) / float(w * h)
    if percentFilled < 0.0005:
        return None
    digit = cv2.bitwise_and(thresh, thresh, mask=mask)
    if debug:
        cv2_imshow(digit)
        cv2.waitKey(0)
    return digit

In [ ]:
image_path = "/content/drive/MyDrive/sudoku_img.jpeg"
model_path = "/content/drive/MyDrive/your_model.h5"

def remove_top_left_parts(image, top_percentage, left_percentage):
    height, width = image.shape[:2]
    cut_top = int(height * top_percentage / 100)
    cut_left = int(width * left_percentage / 100)
    bottom_part = image[cut_top:, cut_left:]
    return bottom_part

model = load_model(model_path)
image = cv2.imread(image_path)
image = imutils.resize(image, width=600)
(puzzleImage, warped) = find_puzzle(image, debug=False)
board = np.zeros((9, 9), dtype="int")
stepX = warped.shape[1] // 9
stepY = warped.shape[0] // 9
cellLocs = []
for y in range(0, 9):
    row = []
    for x in range(0, 9):
        startX = x * stepX
        startY = y * stepY
        endX = (x + 1) * stepX
        endY = (y + 1) * stepY
        row.append((startX, startY, endX, endY))
        cell = warped[startY:endY, startX:endX]
        digit = extract_digit(cell, debug=False)
        if digit is not None:
            roi1=remove_top_left_parts(digit,10,20)
            roi = cv2.resize(roi1, (28, 28))
            cv2_imshow(roi)
            cv2.waitKey(0)
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            pred = model.predict(roi).argmax(axis=1)[0]
            board[y, x] = pred
    cellLocs.append(row)
print("[INFO] OCR'd Sudoku board:")
for row in board:
    print(row)
def isSafe(grid, row, col, num):
    for x in range(9):
        if grid[row][x] == num or grid[x][col] == num:
            return False
    startRow = row - row % 3
    startCol = col - col % 3
    for i in range(3):
        for j in range(3):
            if grid[i + startRow][j + startCol] == num:
                return False
    return True

def solveSudoku(grid, row, col):
    if row == 8 and col == 9:
        return True
    if col == 9:
        row += 1
        col = 0
    if grid[row][col] > 0:
        return solveSudoku(grid, row, col + 1)
    for num in range(1, 10, 1):
        if isSafe(grid, row, col, num):
            grid[row][col] = num
            if solveSudoku(grid, row, col + 1):
                return True
        grid[row][col] = 0
    return False
solved_board = np.copy(board)

if solveSudoku(solved_board, 0, 0):
    print("[INFO] Sudoku puzzle solved:")
    for row in solved_board:
        print(row)
else:
    print("[INFO] No solution found for Sudoku puzzle.")

for (cellRow, boardRow) in zip(cellLocs, solved_board):
    for (box, digit) in zip(cellRow, boardRow):
        startX, startY, endX, endY = box
        textX = int((endX - startX) * 0.33)
        textY = int((endY - startY) * -0.2)
        textX += startX
        textY += endY
        cv2.putText(puzzleImage, str(digit), (textX, textY),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)